* used to avoid overfitting during NN training.

* We need to add dropout layers for avoiding overfitting

* 0.5 = 50 % neurons dropped in the next layer

* It is also trial and error

In [2]:
# UCI Machine Learning Repository

train a network to discriminate between sonar signals bounced off a metal cylinder(M) and those bounced off a roughly cylindrical rock(R)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import warnings
warnings.filterwarnings('ignore') # Ignore warning messages

In [5]:
df = pd.read_csv("/content/sonar_dataset.csv", header=None) # there is no heders, use batana padega, nhi to first row header bn jayegi
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
132,0.0968,0.0821,0.0629,0.0608,0.0617,0.1207,0.0944,0.4223,0.5744,0.5025,...,0.0073,0.0081,0.0303,0.0190,0.0212,0.0126,0.0201,0.0210,0.0041,M
39,0.0091,0.0213,0.0206,0.0505,0.0657,0.0795,0.0970,0.0872,0.0743,0.0837,...,0.0112,0.0112,0.0102,0.0026,0.0097,0.0098,0.0043,0.0071,0.0108,R
52,0.0087,0.0046,0.0081,0.0230,0.0586,0.0682,0.0993,0.0717,0.0576,0.0818,...,0.0052,0.0038,0.0079,0.0114,0.0050,0.0030,0.0064,0.0058,0.0030,R
48,0.0190,0.0038,0.0642,0.0452,0.0333,0.0690,0.0901,0.1454,0.0740,0.0349,...,0.0093,0.0118,0.0112,0.0094,0.0140,0.0072,0.0022,0.0055,0.0122,R
51,0.0131,0.0068,0.0308,0.0311,0.0085,0.0767,0.0771,0.0640,0.0726,0.0901,...,0.0062,0.0028,0.0040,0.0075,0.0039,0.0053,0.0013,0.0052,0.0023,R


In [6]:
df.shape

(208, 61)

In [7]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [8]:
df.isnull().sum().sum()

0

In [9]:
df.columns # these are our column names this time. As already column names/headers were not specied

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [10]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

Hence we got to knoe binary classification problem

In [11]:
df.dtypes

0     float64
1     float64
2     float64
3     float64
4     float64
       ...   
56    float64
57    float64
58    float64
59    float64
60     object
Length: 61, dtype: object

In [12]:
X = df.drop(60,axis='columns')
Y = df[60]

In [13]:
X

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


In [14]:
Y  # it has text data : need to convert into 0,1 integers

0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

In [15]:
Y = pd.get_dummies(Y,drop_first=True)
Y

,R
0,1
1,1
2,1
3,1
4,1
...,...
203,0
204,0
205,0
206,0


This one hot encoding is exact equal to label encoding. Bs difference label tk pahunche one hot encoding se hain. Ye vala tarika we can use in all binary classification problems

The line of code you provided is using one-hot encoding with Pandas `get_dummies`**function to convert categorical variable(s) into dummy/indicator variables.** The `drop_first=True` **parameter indicates that the first level of each categorical variable should be dropped, which helps in preventing multicollinearity in the dataset**.

Here's what each part of the line does:

- `pd.get_dummies(Y)`: This function is used to one-hot encode the categorical variables in `Y`.
- `drop_first=True`: This parameter drops the first level of each categorical variable, which is useful for avoiding the dummy variable trap and reducing multicollinearity.

In summary, this line is preparing the target variable `Y` for a machine learning model, particularly in scenarios where the algorithm being used (e.g., logistic regression) requires numerical input rather than categorical.

In [16]:
Y.value_counts()

R
0    111
1     97
dtype: int64

M = 0, R= 1

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1)

In [20]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


In [21]:
x_train.shape, x_test.shape

((156, 60), (52, 60))

In [22]:
import tensorflow as tf
from tensorflow import keras

In [23]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60,activation='relu'), # this contains input + hidden
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(15,activation = 'relu'), # total 3 hidden and 1 input layer . Neche vali output layer hai
    keras.layers.Dense(1,activation='sigmoid') # output layer
])
#For a binary classification problem, having a single neuron in the output layer with a sigmoid activation function is correct.

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, epochs =100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 4ms/step - loss: 0.7004 - accuracy: 0.3974
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6810 - accuracy: 0.5192
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6653 - accuracy: 0.6026
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6430 - accuracy: 0.6538
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6215 - accuracy: 0.6731
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5801 - accuracy: 0.7244
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5441 - accuracy: 0.7436
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5161 - accuracy: 0.7756
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4925 - accuracy: 0.7628
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4482 - accuracy: 0.8077
Epoch 11/

Jan bujhkar overfit karvaya gya model

In [24]:
# test performance accuracy
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 12ms/step - loss: 1.1233 - accuracy: 0.7500


[1.1232870817184448, 0.75]

As training accuracy 100 % and test accuracy 75 %  => overfitted model

In [25]:
y_pred = model.predict(x_test).reshape(-1) # flattening
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 7ms/step
[2.5805562e-09 9.9607861e-01 9.8750043e-01 2.1976684e-06 9.9997669e-01
 9.9853760e-01 7.1599614e-03 9.9999374e-01 9.5530550e-07 9.9998868e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


The .reshape(-1) operation in NumPy is used to infer the size of one dimension while preserving the total number of elements in the array. In other words, it adjusts the shape of the array to a one-dimensional array (flattens it) while automatically determining the size of that dimension based on the total number of elements in the array.

In [26]:
y_test[:10]

,R
186,0
155,0
165,0
200,0
58,1
34,1
151,0
18,1
202,0
62,1


Model is pretty good, not too bad

In [27]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.89      0.79        27
           1       0.83      0.60      0.70        25

    accuracy                           0.75        52
   macro avg       0.77      0.74      0.74        52
weighted avg       0.77      0.75      0.74        52



# Model with dropout layers after each/some hidden layers

In [29]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1,activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

modeld.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 6s 5ms/step - loss: 0.7300 - accuracy: 0.5321
Epoch 2/100
20/20 [==============================] - 0s 7ms/step - loss: 0.7339 - accuracy: 0.4936
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.7068 - accuracy: 0.5256
Epoch 4/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6812 - accuracy: 0.5705
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6993 - accuracy: 0.4679
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6927 - accuracy: 0.5321
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6745 - accuracy: 0.5449
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6940 - accuracy: 0.5577
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6894 - accuracy: 0.5385
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.6995 - accuracy: 0.4808
Epoch 11/

using dropout reducess the accuracy on training set, but it's still desent. You care more about the test set

In [30]:
modeld.evaluate(x_test,y_test)

2/2 [==============================] - 0s 17ms/step - loss: 0.4547 - accuracy: 0.7885


[0.4547025263309479, 0.7884615659713745]

not much cahnge. But ideally the performance shoud be increased on test data, while using dropout layers

In [33]:
y_pred = modeld.predict(x_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 5ms/step
[5.9890363e-04 8.5809267e-01 8.5614985e-01 1.8327758e-02 9.9905217e-01
 9.0020001e-01 3.6448178e-01 9.9859285e-01 4.0150926e-02 9.9954718e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [34]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.81        27
           1       0.85      0.68      0.76        25

    accuracy                           0.79        52
   macro avg       0.80      0.78      0.78        52
weighted avg       0.80      0.79      0.79        52

